In [3]:
! apt-get update
! apt-get install git -y

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease                       
Hit:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease               
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  git-man krb5-locales less libbrotli1 libbsd0 libcbor0.6 libcurl3-gnutls
  libedit2 liberror-perl libexpat1 libfido2-1 libgssapi-krb5-2 libk5crypto3
  libkeyutils1 libkrb5-3 libkrb5support0 libnghttp2-14 libpsl5 librtmp1
  libssh-4 libx11-6 libx11-data libxau6 libxcb1 libxdmcp6 libxext6 libxmuu1
  openssh-client publicsuffix xauth
Suggested packages:
  gettext-base git-daemon-run | git-daemon-sysvinit git-doc git-el git-email
  git-gui gi

In [4]:
# Base Library
!pip install datasets -U
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install pandas
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 8.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 28.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 58.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━

In [7]:
!pip install auto_gptq


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import torch
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from transformers import TextGenerationPipeline
from transformers import AutoTokenizer

/home/jovyan/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
CUDA extension not installed.
CUDA extension not installed.


In [2]:
pretrained_model_name = "beomi/Llama-3-Open-Ko-8B" 
quantize_config = BaseQuantizeConfig(bits=4, group_size=128)

# Load Model
model = AutoGPTQForCausalLM.from_pretrained(
    pretrained_model_name,
    quantize_config=quantize_config,
    trust_remote_code=False,
    device_map="auto",
    torch_dtype=torch.float16
)
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

/home/jovyan/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 6/6 [01:46<00:00, 17.69s/it]


In [ ]:
example_text = "안녕하세요 저의 이름은 이승유 입니다."
tokenized_input = tokenizer(example_text, return_tensors="pt").to("cuda")  # 'return_tensors="pt"'로 PyTorch 텐서 생성 후 GPU로 전송

# Quantize 모델
model.to("cuda")  # 모델을 GPU로 보냄
model.quantize([tokenized_input])  # 리스트 형태로 quantize에 전달

INFO - Start quantizing layer 1/32
INFO - Quantizing self_attn.k_proj in layer 1/32...
INFO - Quantizing self_attn.v_proj in layer 1/32...
INFO - Quantizing self_attn.q_proj in layer 1/32...
INFO - Quantizing self_attn.o_proj in layer 1/32...
INFO - Quantizing mlp.up_proj in layer 1/32...
INFO - Quantizing mlp.gate_proj in layer 1/32...
INFO - Quantizing mlp.down_proj in layer 1/32...
INFO - Start quantizing layer 2/32
INFO - Quantizing self_attn.k_proj in layer 2/32...
INFO - Quantizing self_attn.v_proj in layer 2/32...
INFO - Quantizing self_attn.q_proj in layer 2/32...
INFO - Quantizing self_attn.o_proj in layer 2/32...
INFO - Quantizing mlp.up_proj in layer 2/32...
INFO - Quantizing mlp.gate_proj in layer 2/32...
INFO - Quantizing mlp.down_proj in layer 2/32...
INFO - Start quantizing layer 3/32
INFO - Quantizing self_attn.k_proj in layer 3/32...
INFO - Quantizing self_attn.v_proj in layer 3/32...
INFO - Quantizing self_attn.q_proj in layer 3/32...
INFO - Quantizing self_attn.o_pro

In [ ]:
quantized_model_dir = "Ko-Llama-quantized"
model.save_quantized(quantized_model_dir)

In [ ]:
# Inference

device = "cuda:0"  # make use of GPU for inference.
model = AutoGPTQForCausalLM.from_quantized(quantized_model_dir, device=device, torch_dtype=torch.float16)

pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer, max_new_tokens=50)  

print(pipeline("안녕하세요")[0]["generated_text"])

In [ ]:
# Another Code for Quantization
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig

model_id = "beomi/Llama-3-Open-Ko-8B" 

tokenizer = AutoTokenizer.from_pretrained(model_id)


quantization_config = GPTQConfig(bits=4, dataset = "c4", tokenizer=tokenizer)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=quantization_config)

In [3]:
! pip install autoawq

INFO: pip is looking at multiple versions of autoawq-kernels to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 1.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 15.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 12.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 26.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from awq import AutoAWQForCausalLM
from datasets import load_dataset

pretrained_model_name = 'saltlux/Ko-Llama3-Luxia-8B'

model = AutoAWQForCausalLM.from_pretrained(pretrained_model_name, device_map="auto")

my_quant_config = {"zero_point" : True,
                   "q_group_size":64,   # Use 128 group size for better performance
                   "w_bit":4,   # INT4 quantization
                   'version': 'GEMM'
                  }



/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:54<00:00, 13.70s/it]


In [5]:

"""
"gemm" 버전:

scales와 zeros를 전치(transpose)하고 contiguous 함수로 메모리를 연속적으로 재배열한 후,
WQLinear_GEMM 모듈을 사용합니다.
이는 일반적인 행렬 연산(general matrix multiplication, GEMM)을 사용한 버전으로 볼 수 있습니다.

"gemv" 버전:
WQLinear_GEMV 모듈을 사용합니다.
GEMV는 벡터화된 행렬 곱셈(general matrix-vector multiplication)으로, 메모리 최적화 및 속도 면에서 유리합니다.

"marlin" 버전:
WQLinear_Marlin 모듈을 사용합니다.
이 버전은 특수한 하드웨어나 특정 목적을 위한 커스텀 최적화가 이루어진 버전

"gemv_fast" 버전:
WQLinear_GEMVFast 모듈을 사용하며, gemv와 유사하나 빠른 연산을 위해 추가적인 최적화를 포함한 버전입니다.
"""

'\n"gemm" 버전:\n\nscales와 zeros를 전치(transpose)하고 contiguous 함수로 메모리를 연속적으로 재배열한 후,\nWQLinear_GEMM 모듈을 사용합니다.\n이는 일반적인 행렬 연산(general matrix multiplication, GEMM)을 사용한 버전으로 볼 수 있습니다.\n\n"gemv" 버전:\nWQLinear_GEMV 모듈을 사용합니다.\nGEMV는 벡터화된 행렬 곱셈(general matrix-vector multiplication)으로, 메모리 최적화 및 속도 면에서 유리합니다.\n\n"marlin" 버전:\nWQLinear_Marlin 모듈을 사용합니다.\n이 버전은 특수한 하드웨어나 특정 목적을 위한 커스텀 최적화가 이루어진 버전\n\n"gemv_fast" 버전:\nWQLinear_GEMVFast 모듈을 사용하며, gemv와 유사하나 빠른 연산을 위해 추가적인 최적화를 포함한 버전입니다.\n'

In [6]:
def load_calibration_data():
    """Load calibration data for quantization"""
    data = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train')
    return [text for text in data['text'] if text.strip() != '' and len(text.split(' ')) > 20]

# Main function to quantize the model

In [7]:

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

calibration_data=load_calibration_data()

model.eval()


Generating validation split: 100%|██████████| 3760/3760 [00:00<00:00, 62779.23 examples/s]


LlamaAWQForCausalLM(
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(145792, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
          (post_atten

In [8]:

model.quantize(
    tokenizer,
    quant_config=my_quant_config,
    calib_data=calibration_data,  # Use calibration data
    n_parallel_calib_samples=1,  # Process one sample at a time to avoid memory overload
    max_calib_samples=10,  # Reduce calibration sample size for memory
)


AWQ: 100%|██████████| 32/32 [45:57<00:00, 86.17s/it]


In [ ]:
save_dir='awq_dir'

model.save(save_dir, safetensors=True , shard_size="10GB")

tokenizer.save_pretrained(save_dir)